이 파일은 colab 기준으로 작성되었습니다.

#음성 전처리

In [ ]:
!pip install webrtcvad soundfile scipy
!pip install faster-whisper

##Google Drive Mount

In [ ]:
# === Google Drive Mount ===
from google.colab import drive
drive.mount('/content/drive')

##m4a → wav 변환(16kHz, mono)

In [ ]:
# === 0) ffmpeg 변환 (모든 포맷 → 16kHz / mono WAV) ===
import subprocess

# 원본 오디오 파일 경로 (예: m4a, mp3 등)
audio_path = "파일 경로"
# 변환 결과 (전처리 입력용)
converted_wav = "파일 경로"

print("🔄 ffmpeg 변환 중... (16kHz / mono)")
subprocess.run([
    "ffmpeg", "-y",
    "-i", audio_path,
    "-ar", "16000",   # 16kHz
    "-ac", "1",       # mono
    "-acodec", "pcm_s16le",
    converted_wav
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
print(f"✅ 변환 완료 → {converted_wav}")

# 이후 input_wav 경로를 ffmpeg 변환된 파일로 지정
input_wav = converted_wav

##전처리 파라미터, 함수 정의

In [ ]:
# === Whisper 투입 전 전처리 파이프라인 ===
# 입력: 16kHz / mono WAV (m4a→wav 변환은 이미 끝난 상태)
# 단계: HPF(80Hz) -> VAD(webrtcvad) -> Gain Normalize -> 저장

import os
import numpy as np
import soundfile as sf
import webrtcvad
from scipy.signal import butter, sosfiltfilt

In [ ]:
# -----------------------------
# 설정값
# -----------------------------
# 위에서 생성된 converted_wav 파일을 input_wav로 사용
output_wav = "파일 경로"

SR = 16000
HPF_CUTOFF = 80.0      # Hz
HPF_ORDER  = 4         # Butterworth 차수
VAD_AGGR   = 2         # 0(느슨) ~ 3(공격적) 권장: 2
FRAME_MS   = 20        # webrtcvad 허용: 10/20/30ms
PAD_MS     = 150       # 발화 구간 전후 패딩 (hangover)
TARGET_PEAK = 0.98     # Peak 정규화 목표치 (0~1), 너무 높이면 clip 위험

In [ ]:
# -----------------------------
# 유틸 함수
# -----------------------------
def highpass_hz_80(audio: np.ndarray, sr: int) -> np.ndarray:
    sos = butter(HPF_ORDER, HPF_CUTOFF, btype="highpass", fs=sr, output="sos")
    return sosfiltfilt(sos, audio).astype(np.float32)

def float_to_int16(x: np.ndarray) -> np.ndarray:
    x = np.clip(x, -1.0, 1.0)
    return (x * 32767.0).astype(np.int16)

def int16_to_float(x: np.ndarray) -> np.ndarray:
    return (x.astype(np.float32) / 32767.0)

def frame_bytes_from_int16(x_i16: np.ndarray, sr: int, frame_ms: int):
    frame_len = int(sr * frame_ms / 1000)
    pad = (-len(x_i16)) % frame_len
    if pad:
        x_i16 = np.pad(x_i16, (0, pad), mode="constant")
    for i in range(0, len(x_i16), frame_len):
        yield x_i16[i:i+frame_len].tobytes()
    return pad

def vad_keep_mask(audio_f32: np.ndarray, sr: int, frame_ms: int, aggressiveness: int, pad_ms: int):
    vad = webrtcvad.Vad(aggressiveness)
    x_i16 = float_to_int16(audio_f32)
    frame_len = int(sr * frame_ms / 1000)
    n_frames = int(np.ceil(len(x_i16) / frame_len))
    pad = (-len(x_i16)) % frame_len
    if pad:
        x_i16 = np.pad(x_i16, (0, pad), mode="constant")
    voiced = np.zeros(n_frames, dtype=bool)

    for idx in range(n_frames):
        frm = x_i16[idx*frame_len:(idx+1)*frame_len].tobytes()
        voiced[idx] = vad.is_speech(frm, sr)

    pad_frames = max(1, int(pad_ms / frame_ms))
    keep = np.zeros_like(voiced, dtype=bool)
    for i, v in enumerate(voiced):
        if v:
            s = max(0, i - pad_frames)
            e = min(n_frames, i + pad_frames + 1)
            keep[s:e] = True

    keep_samples = np.repeat(keep, frame_len)
    keep_samples = keep_samples[:len(x_i16)]
    keep_samples = keep_samples[:len(audio_f32)]
    if len(keep_samples) < len(audio_f32):
        keep_samples = np.pad(keep_samples, (0, len(audio_f32)-len(keep_samples)), mode="constant")

    return keep_samples

def peak_normalize(x: np.ndarray, target_peak: float = 0.98) -> np.ndarray:
    peak = np.max(np.abs(x)) + 1e-12
    g = target_peak / peak
    y = np.clip(x * g, -1.0, 1.0)
    return y.astype(np.float32)

##전처리 과정 진행

In [ ]:
# -----------------------------
# 파이프라인 실행
# -----------------------------
# 1) 읽기 (16kHz/mono 가정)
audio, sr = sf.read(input_wav, always_2d=False)
if sr != SR:
    raise ValueError(f"입력 SR={sr}가 16kHz가 아닙니다. 변환 후 파일을 넣어주세요.")
if audio.ndim != 1:
    raise ValueError("입력이 mono가 아닙니다. 변환 후 mono WAV를 넣어주세요.")
audio = audio.astype(np.float32)

# 2) HPF(80Hz)
audio_hpf = highpass_hz_80(audio, SR)

# 3) VAD로 발화 구간만 유지(전후 PAD)
mask = vad_keep_mask(audio_hpf, SR, FRAME_MS, VAD_AGGR, PAD_MS)
voiced = audio_hpf[mask]

if voiced.size < int(0.1 * len(audio_hpf)) and len(audio_hpf) > 0:
    voiced = audio_hpf  # fallback

# 4) Gain Normalization (peak 기준)
voiced_norm = peak_normalize(voiced, TARGET_PEAK)

# 5) 저장 (PCM_16)
os.makedirs(os.path.dirname(output_wav), exist_ok=True)
sf.write(output_wav, voiced_norm, SR, subtype="PCM_16")

print(f"[OK] 저장 완료 → {output_wav}")
print(f"원본 길이: {len(audio)/SR:.2f}s,  전처리 후 길이: {len(voiced_norm)/SR:.2f}s")

#Chunk 분할

In [ ]:
# ===============================================
# ✅ OpenAI Whisper-1 한국어 전사 + txt 저장 (Colab용)
# ===============================================

# 1️⃣ 라이브러리 설치
!pip install --upgrade openai
!pip install --upgrade openai pydub
!apt -q install -y ffmpeg

In [ ]:
from pathlib import Path
from pydub import AudioSegment
import math
import subprocess

# ===== 설정 =====
AUDIO_PATH = Path("입력 파일 위치")   # 원본 오디오
CHUNK_MINUTES = 10                                               # 10분 단위
DRIVE_CHUNK_DIR = Path("출력 파일 위치")                         # 전송용 wav 저장 폴더(Drive)
LOCAL_TMP_DIR = Path("/content/chunks_tmp")                      # 로컬 임시 폴더(WAV 등)

SAMPLE_RATE = 16000
MAX_TARGET_MB = 25  # 안내용 체크 (실제 한도는 25MB)

# ===== 준비 =====
assert AUDIO_PATH.exists(), f"원본 오디오가 없습니다: {AUDIO_PATH}"
DRIVE_CHUNK_DIR.mkdir(parents=True, exist_ok=True)
LOCAL_TMP_DIR.mkdir(parents=True, exist_ok=True)

# ===== 분할 =====
audio = AudioSegment.from_file(AUDIO_PATH)
chunk_ms = CHUNK_MINUTES * 60 * 1000
num_chunks = math.ceil(len(audio) / chunk_ms)
print(f"🔹 총 {num_chunks}개로 분할 예정(약 {CHUNK_MINUTES}분 단위)")

exported = []
for i, start in enumerate(range(0, len(audio), chunk_ms)):
    seg = audio[start:start+chunk_ms]

    # ★ 로컬 임시 wav (원본 유지)
    raw_wav = LOCAL_TMP_DIR / f"chunk_{i:04d}.wav"
    seg.export(raw_wav, format="wav")

    # ============================================
    # ★★★ ffmpeg로 WAV 그대로 16kHz mono로 저장
    # ============================================
    wav_path = DRIVE_CHUNK_DIR / f"chunk_{i:04d}.wav"
    cmd = [
        "ffmpeg", "-v", "error", "-y",
        "-i", str(raw_wav),
        "-ac", "1", "-ar", str(SAMPLE_RATE),
        str(wav_path),
    ]
    subprocess.run(cmd, check=True)

    size_mb = wav_path.stat().st_size / (1024 * 1024)
    print(f"  ▶ chunk_{i:04d}.wav  {size_mb:.2f} MB")
    if size_mb >= MAX_TARGET_MB:
        print("    ⚠️ 25MB에 근접/초과합니다. 필요 시 CHUNK_MINUTES를 더 줄이세요.")

    exported.append(wav_path)

print(f"✅ WAV 분할 완료: {len(exported)}개 파일 → {DRIVE_CHUNK_DIR}")

#Whisper

In [ ]:
from pathlib import Path

# ✅ 분할된 청크(wav)가 들어있는 폴더 (앞서 사용하던 경로 그대로)
CHUNK_DIR = Path("청크 위치")

# 출력 폴더들
SRT_DIR = Path("SRT 저장 위치")   # 청크별 SRT
TXT_DIR = Path("TXT 저장 위치")   # 청크별 타임스탬프 TXT
FINAL_TXT = Path("최종 TXT 저장 위치")  # 최종 1개 TXT

SRT_DIR.mkdir(parents=True, exist_ok=True)
TXT_DIR.mkdir(parents=True, exist_ok=True)

# 정렬된 청크 목록
chunk_files = sorted(CHUNK_DIR.glob("chunk_*.wav"))
assert chunk_files, f"청크 wav가 없습니다: {CHUNK_DIR}"
print(f"청크 {len(chunk_files)}개 발견")
for p in chunk_files:
    print(" -", p.name)

In [ ]:
import os
# 이곳에 open ai api key 입력
os.environ["OPENAI_API_KEY"] = "Open AI API Key"

In [ ]:
# ===============================================
from openai import OpenAI
from pathlib import Path

client = OpenAI()

for i, cp in enumerate(chunk_files, 1):
    size_mb = cp.stat().st_size / (1024*1024)
    print(f"▶️ {i}/{len(chunk_files)} SRT 전사: {cp.name} ({size_mb:.2f} MB)")
    with cp.open("rb") as f:
        srt_text = client.audio.transcriptions.create(
            model="whisper-1",
            file=f,
            language="ko",     # 한국어 고정 원하면 주석 해제
            response_format="srt"
        )
    (SRT_DIR / f"{cp.stem}.srt").write_text(srt_text, encoding="utf-8")
print("✅ 모든 청크 SRT 전사 완료")

In [ ]:
from pydub import AudioSegment
from datetime import timedelta
import re

# ---------- 유틸: 시간 변환 ----------
def ms_to_srt_time(ms:int)->str:
    td = timedelta(milliseconds=ms)
    hours, rem = divmod(td.seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    hours += td.days * 24
    millis = int(ms % 1000)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}.{millis:03d}"

def srt_time_to_ms(t:str)->int:
    # SRT: "HH:MM:SS,mmm"
    h, m, rest = t.split(":")
    s, ms = rest.split(",")
    return (int(h)*3600 + int(m)*60 + int(s))*1000 + int(ms)

# ---------- 유틸: SRT 파서 ----------
# "00:00:01,000 --> 00:00:04,000" 줄과 본문을 cue로 파싱
re_time = re.compile(r"(\d\d:\d\d:\d\d,\d\d\d)\s*-->\s*(\d\d:\d\d:\d\d,\d\d\d)")

def parse_srt(srt_text:str):
    lines = [ln.rstrip("\n") for ln in srt_text.splitlines()]
    cues = []
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        # 번호 라인(있을 수도 없음)
        if line.isdigit():
            i += 1
            if i >= len(lines): break
            line = lines[i].strip()
        # 시간 라인
        m = re_time.match(line)
        if not m:
            i += 1
            continue
        start, end = m.group(1), m.group(2)
        i += 1
        # 본문
        text_lines = []
        while i < len(lines) and lines[i].strip():
            text_lines.append(lines[i])
            i += 1
        cues.append((start, end, "\n".join(text_lines)))
        # 빈 줄 스킵
        while i < len(lines) and not lines[i].strip():
            i += 1
    return cues

# ---------- 1) 청크 누적 오프셋 계산 ----------
offsets = []
acc = 0
for cp in chunk_files:
    offsets.append(acc)
    dur_ms = len(AudioSegment.from_file(cp))
    acc += dur_ms
print("누적 오프셋(ms):", offsets)

# ---------- 2) 청크별 TXT 생성 + 최종 병합 ----------
all_lines = []
for idx, cp in enumerate(chunk_files):
    srt_path = SRT_DIR / f"{cp.stem}.srt"
    assert srt_path.exists(), f"SRT 없음: {srt_path}"
    srt_text = srt_path.read_text(encoding="utf-8")
    cues = parse_srt(srt_text)
    off = offsets[idx]

    # 청크별 TXT(옵션)
    per_chunk_txt = []
    for (st, et, text) in cues:
        st_ms = srt_time_to_ms(st) + off
        et_ms = srt_time_to_ms(et) + off
        one_line = f"[{ms_to_srt_time(st_ms)} - {ms_to_srt_time(et_ms)}] {text.replace('\n',' ')}"
        per_chunk_txt.append(one_line)
        all_lines.append(one_line)

    (TXT_DIR / f"{cp.stem}.txt").write_text("\n".join(per_chunk_txt), encoding="utf-8")
    print(f"✅ 청크 TXT 저장: {(TXT_DIR / f'{cp.stem}.txt').name}")

# 최종 병합본 저장
FINAL_TXT.write_text("\n".join(all_lines), encoding="utf-8")
print(f"🎯 최종 타임스탬프 TXT 병합 완료 → {FINAL_TXT}")

#후처리 - 중복 문장 제거

In [ ]:
from pathlib import Path
import re
from typing import List, Tuple

# ====== 경로 설정 ======
INPUT_TXT = Path("입력 파일 위치")  # 입력
OUTPUT_TXT = Path("출력 파일 위치")  # 출력

assert INPUT_TXT.exists(), f"입력 파일 없음: {INPUT_TXT}"

# ====== 라인 파서: [HH:MM:SS.mmm - HH:MM:SS.mmm] 텍스트 ======
re_line = re.compile(
    r"^\[(\d{2}:\d{2}:\d{2}\.\d{3})\s*-\s*(\d{2}:\d{2}:\d{2}\.\d{3})\]\s*(.*)$"
)

def parse_line(line: str) -> Tuple[str, str, str]:
    m = re_line.match(line.strip())
    if not m:
        # 형식에서 벗어난 라인은 내용 전체를 text로 간주(타임스탬프는 빈 값)
        return ("", "", line.strip())
    return m.group(1), m.group(2), m.group(3)

# ====== 정규화 유틸 ======
def normalize_text(s: str) -> str:
    # 공백 정리 + 끝 공백 제거
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ====== 라인 내부 중복 축약 ======
def collapse_sentence_runs(text: str) -> str:
    """
    문장부호 기준으로 문장을 나눠 연속 동일 문장 중복을 1회로 축약.
    """
    # 문장 단위 분리(구두점 유지)
    parts = re.split(r"([\.!\?…]+)", text)
    # parts = [frag, punct, frag, punct, ... , last_frag?]
    merged = []
    last_sent_norm = None

    i = 0
    while i < len(parts):
        frag = parts[i].strip()
        punct = parts[i+1] if i+1 < len(parts) else ""
        sent = (frag + punct).strip()
        if sent:
            curr_norm = normalize_text(frag.lower())
            if curr_norm != last_sent_norm:
                merged.append(sent)
                last_sent_norm = curr_norm
            # 동일 문장 연속이면 skip
        i += 2
    return normalize_text(" ".join(merged))

def collapse_word_runs(text: str) -> str:
    """
    같은 단어가 연속으로 반복될 때 1회로 축약.
    (공백 기준 단순 토큰화; 한국어 띄어쓰기 기준)
    """
    toks = text.split()
    out = []
    last = None
    for t in toks:
        if last is None or t != last:
            out.append(t)
            last = t
        # 동일 토큰이면 skip
    return " ".join(out)

def dedup_inside_line(text: str) -> str:
    # 1) 문장 반복 축약 → 2) 단어 반복 축약 → 3) 공백 정리
    s = collapse_sentence_runs(text)
    s = collapse_word_runs(s)
    s = normalize_text(s)
    return s

# ====== 파일 로드 & 파싱 ======
raw_lines = INPUT_TXT.read_text(encoding="utf-8", errors="ignore").splitlines()
entries = []
for ln in raw_lines:
    st, et, tx = parse_line(ln)
    tx = dedup_inside_line(tx)
    entries.append({"start": st, "end": et, "text": tx})

# ====== 라인 간 중복/포함 제거 ======
def norm_for_compare(s: str) -> str:
    # 비교용 노멀라이즈(소문자/공백 정리/문장 끝 점 하나로 통일)
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"(\.){2,}$", ".", s)  # 끝 점 연속은 1개로
    return s

kept = []
removed = 0

for i, cur in enumerate(entries):
    cur_txt = cur["text"]
    if not cur_txt:
        # 빈 텍스트는 제거
        removed += 1
        continue

    cur_norm = norm_for_compare(cur_txt)

    # 미리 다음 라인 텍스트를 확인(현재가 더 짧고 포함되면 지금 삭제)
    next_norm = None
    if i + 1 < len(entries):
        nxt_txt = entries[i+1]["text"]
        next_norm = norm_for_compare(nxt_txt) if nxt_txt else None

    # 규칙 2-1) 현재 ⊆ 다음(또는 동일) 이고 현재 길이 <= 다음이면 현재 삭제
    if next_norm and cur_norm and (cur_norm == next_norm or cur_norm in next_norm):
        if len(cur_norm) <= len(next_norm):
            removed += 1
            continue  # drop current, keep next

    # 이전과 비교 (kept의 마지막과만 비교)
    if kept:
        prev = kept[-1]
        prev_norm = norm_for_compare(prev["text"])

        # 동일/포함 관계 → 짧은 쪽 삭제, 같으면 뒤쪽(현재) 삭제
        if cur_norm == prev_norm:
            removed += 1
            continue  # drop current (뒤쪽)
        if cur_norm in prev_norm:
            removed += 1
            continue  # current 더 짧음 → drop current
        if prev_norm in cur_norm:
            # prev 더 짧음 → prev 삭제, current 채택
            kept.pop()
            kept.append(cur)
            continue

    # 여기까지 왔으면 채택
    kept.append(cur)

# ====== 저장 ======
out_lines = []
for e in kept:
    st = e["start"] or "00:00:00.000"
    et = e["end"] or "00:00:00.000"
    out_lines.append(f"[{st} - {et}] {e['text']}")

OUTPUT_TXT.write_text("\n".join(out_lines), encoding="utf-8")

print(f"입력 라인 수: {len(entries)}")
print(f"제거 라인 수: {removed}")
print(f"남은 라인 수: {len(kept)}")
print(f"✅ 저장 완료 → {OUTPUT_TXT}")